In [10]:
import time
import numpy as np
import matplotlib.pyplot as plt
import cv2

neighbor_size = 2  # width = 2x+1
disparity = 1
method = 'ssd'

In [11]:

# file = 'moebius'
file = 'tsukuba'
if file == 'moebius':
    file1 = 'moebius1.png'
    file2 = 'moebius2.png'
else:
    file1 = 'tsukuba1.jpg'
    file2 = 'tsukuba2.jpg'

file_name1 = 'data/{}'.format(file1)
file_name2 = 'data/{}'.format(file2)
input_left = cv2.imread(file_name1, 0)
input_right = cv2.imread(file_name2, 0)

height, width = input_left.shape
print(input_left.shape)

(288, 384)


In [12]:
def descriptor_range(cur_row, cur_col, max_row, max_col, neighbor_size):
    row_range_from = max(0, cur_row - neighbor_size)
    row_range_to = min(max_row - 1, cur_row + neighbor_size)
    col_range_from = max(0, cur_col - neighbor_size)
    col_range_to = min(max_col - 1, cur_col + neighbor_size)
    len = (row_range_to - row_range_from + 1) * (col_range_to - col_range_from + 1)
    return row_range_from, row_range_to, col_range_from, col_range_to, len


def compare_similarity(descriptor_left, descriptor_right, method):
    dis = 0
    if method == 'ssd':
        dis = np.sum((descriptor_left - descriptor_right) * 2) / descriptor_left.shape[1]
    elif method == 'nc':
        x = descriptor_left - np.mean(descriptor_left)
        c = descriptor_right - np.mean(descriptor_right)
        x = x / np.sqrt(np.sum(np.sqrt(x)))
        c = c / np.sqrt(np.sum(np.sqrt(c)))
        dis = x * c
    return dis


corresponding_point = np.zeros((height, width))
depth = np.zeros((height, width))
for i in range(height):
    for j in range(width):
        row_range_from, rwo_range_to, col_range_from, col_range_to, len_left = descriptor_range(i, j, height, width,
                                                                                                neighbor_size)
        descriptor_left = np.reshape(input_left[row_range_from: rwo_range_to + 1, col_range_from:col_range_to + 1],
                                     (1, len_left))
        similarity = []
        index_list = []
        mind = max(-disparity, -col_range_from)
        # mind = 0
        maxd = min(disparity, width - 1 - col_range_to)
        for k in range(mind, maxd + 1):
            row_range_from, rwo_range_to, col_range_from, col_range_to, len_right = descriptor_range(i, k, height,
                                                                                                     width,
                                                                                                     neighbor_size)
            if len_left == len_right:
                descriptor_right = np.reshape(
                    input_left[row_range_from: rwo_range_to + 1, col_range_from:col_range_to + 1],
                    (1, len_right))
                similarity.append(compare_similarity(descriptor_left, descriptor_right, method))
                index_list.append(k)

        index = 0
        if method == 'ssd':
            index = np.argmin(np.array(similarity))
        elif method == 'nc':
            index = np.argmax(np.array(similarity))

        corresponding_point[i, j] = index_list[index]
        depth[i, j] = np.abs(j - corresponding_point[i, j])

# scaled_depth = (depth - np.min(depth)) / (np.max(depth - np.min(depth)))
plt.imshow(depth, cmap='gray_r')

ValueError: attempt to get argmin of an empty sequence